# * Import : "CDSAPPO.DIM_TIME"
    WHERE TM_KEY_YR >= 2023

## Parameter

In [1]:
import os
import glob
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')
curr_dt

datetime.date(2024, 6, 18)

## ETL Process...

### Step 1 : DIM_TIME
    DB -> CSV file

In [5]:
# Input : 0 for dummy parameter

op_dir = 'data'
op_file = 'dim_time.csv'
v_param = dict(yr_start=2023, yr_end=0)

job_start_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nJob Start... {job_start_datetime}')

with open('SQL/Import-DIM_TIME.sql', 'r') as sql_file:
    queries = sql_file.read().split(';')
    query = queries[0].strip()
    sql_file.close()

# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()


try:
    print(f"\nProcessing...\n\n   -> data as of (YR_Start: {v_param['yr_start']}, YR_End: {v_param['yr_end']})")

    # Create Dataframe
    src_cur.execute(query, v_param)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> src_df : {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_file}" successfully')


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')
    print(f'\nJob Done !!!')



Job Start... 2024-06-18, 22:53:16

TDMDBPR : Connected

Processing...

   -> data as of (YR_Start: 2023, YR_End: 0)

   -> src_df : 1096 rows, 23 columns

   -> Generate "dim_time.csv" successfully

TDMDBPR : Disconnected

Job Done !!!


In [6]:
src_df.tail()

,TM_KEY_DAY,DAY_DESC,DAY_SHORT,DAY_NO,DATE_VALUE,DAY_OF_WEEK,DAYS_IN_MONTH,TRUE_TM_KEY_WK,TRUE_WEEK,TM_KEY_WK,...,MONTH_SHORT,TM_KEY_QTR,QUARTER_NO,TM_KEY_YR,YEAR_DESC,YEAR_DESC_TH,PERIODFLAG,ETL_DATE,ETL_UPDATE,LOAD_DATE
1091,20251227,Saturday,Sat,27,2025-12-27,6,31,2025052,52,2025052,...,Dec,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41
1092,20251228,Sunday,Sun,28,2025-12-28,7,31,2025052,52,2025053,...,Dec,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41
1093,20251229,Monday,Mon,29,2025-12-29,1,31,2026001,1,2025053,...,Dec,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41
1094,20251230,Tuesday,Tue,30,2025-12-30,2,31,2026001,1,2025053,...,Dec,20254,4,2025,2025,2568,N,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41
1095,20251231,Wednesday,Wed,31,2025-12-31,3,31,2026001,1,2025053,...,Dec,20254,4,2025,2025,2568,EMQY,2024-01-30 16:04:35,2024-01-30 16:04:35,2024-06-18 22:53:41


In [7]:
src_df.dtypes

TM_KEY_DAY                 int64
DAY_DESC                  object
DAY_SHORT                 object
DAY_NO                     int64
DATE_VALUE        datetime64[ns]
DAY_OF_WEEK                int64
DAYS_IN_MONTH              int64
TRUE_TM_KEY_WK             int64
TRUE_WEEK                  int64
TM_KEY_WK                  int64
WEEK_YEAR                  int64
TM_KEY_MTH                 int64
MONTH_NO                   int64
MONTH_SHORT               object
TM_KEY_QTR                 int64
QUARTER_NO                 int64
TM_KEY_YR                  int64
YEAR_DESC                 object
YEAR_DESC_TH              object
PERIODFLAG                object
ETL_DATE          datetime64[ns]
ETL_UPDATE        datetime64[ns]
LOAD_DATE         datetime64[ns]
dtype: object